In [2]:
import os
import re


In [17]:
# BAM_MAP_LOCATION = '/gscuser/estorrs/CPTAC3.catalog/MGI.BamMap.dat'
# BAM_MAP_LOCATION = '/gscuser/estorrs/gbm.bobo.bammap.dat'
BAM_MAP_LOCATION = '/gscuser/estorrs/cptac2.bam_locations.alla.tsv'

EXECUTION_DIR = '/gscmnt/gc2508/dinglab/estorrs/cptac3/data/ov/hg19'

In [18]:
f = open(BAM_MAP_LOCATION)
sample_data = {}


# identifiers = [r'WXS|WGS', 'hg38', 'BAM', r'tissue_normal|blood_normal', 'CCRCC']
identifiers = [r'WXS|WGS', '.bam', 'DN', 'OV']
# identifiers = ['blood_normal']


# filter based on identifiers and get sample name and location
for line in f:
    is_valid = True
    for identifier in identifiers:
        if not re.findall(identifier, line):
            is_valid = False
            break

    if is_valid:
        pieces = line.strip().split('\t')

        sample = pieces[1]
        technology = pieces[3]
        fp = pieces[5]
        
        if sample in sample_data:
            d = sample_data[sample]
            if d['technology'].lower() != 'wxs' and technology.lower() == 'wxs':
                sample_data[sample] = {'technology': technology, 'fp': fp}
        else:
            sample_data[sample] = {'technology': technology, 'fp': fp}
len(sample_data), list(sample_data.items())[:5]

(102,
 [('01OV002',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/3_DN/analysis/3_DN.sorted.rmdup.bam'}),
  ('01OV007',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/4_DN/analysis/4_DN.sorted.rmdup.bam'}),
  ('01OV008',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/5_DN/analysis/5_DN.sorted.rmdup.bam'}),
  ('01OV010',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/6_DN/analysis/6_DN.sorted.rmdup.bam'}),
  ('01OV013',
   {'technology': 'WXS',
    'fp': '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/7_DN/analysis/7_DN.sorted.rmdup.bam'})])

In [19]:
sample_fp_tups = [(k, d['fp']) for k, d in sample_data.items()]
len(sample_fp_tups), sample_fp_tups[:5]

(102,
 [('01OV002',
   '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/3_DN/analysis/3_DN.sorted.rmdup.bam'),
  ('01OV007',
   '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/4_DN/analysis/4_DN.sorted.rmdup.bam'),
  ('01OV008',
   '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/5_DN/analysis/5_DN.sorted.rmdup.bam'),
  ('01OV010',
   '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/6_DN/analysis/6_DN.sorted.rmdup.bam'),
  ('01OV013',
   '/gscmnt/gc2523/dinglab/cptac/exome_batch1-1/7_DN/analysis/7_DN.sorted.rmdup.bam')])

In [21]:
# write sample to bam location file
sample_to_bam_fp = os.path.join(EXECUTION_DIR, 'sample_to_bam.tsv')
out_f = open(sample_to_bam_fp, 'w')
for sample, fp in sample_fp_tups:
    out_f.write(f'{sample}\t{fp}\n')
out_f.close()

In [22]:
os.mkdir(os.path.join(EXECUTION_DIR, 'bams'))

In [23]:
# write softlink script
softlink_script_fp = os.path.join(EXECUTION_DIR, 'symlink_bams.sh')
output_bam_dir = os.path.join(EXECUTION_DIR, 'bams')
out_f = open(softlink_script_fp, 'w')
for sample, fp in sample_fp_tups:
    out_f.write(f'ln -s {fp} {output_bam_dir}/{sample}.bam\n')
out_f.close()